# Taller de NLP: Word2Vec

In [894]:
#%load_ext watermark
#%watermark -a 'Santiago Alferez' --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Author: Santiago Alferez

numpy     : 1.21.2
pandas    : 1.3.5
re        : 2.2.1
nltk      : 3.6.7
sklearn   : 1.0.2
matplotlib: 3.5.0
scipy     : 1.7.3
gensim    : 4.0.1



Para este taller deberás disponer de algunas librerías como scikit-learn, NLTK, y GenSim. Se recomienda revisar la [documentación de GenSim](https://radimrehurek.com/gensim/auto_examples/index.html#documentation).

## Cargando un modelo  en GenSim y análisis
A continuación cargaremos un modelo que no pesa tanto `glove-twitter-50`. Hay modelos más completos y con mayor número de dimensiones en este [link](https://github.com/RaRe-Technologies/gensim-data)

In [1]:
import gensim.downloader as api

In [2]:
wv = api.load("glove-twitter-50")

Una operación común es recuperar el vocabulario de un modelo. Eso es trivial:

In [52]:
for index, word in enumerate(wv.index_to_key):
    if index == 20:
        break
    print(f"palabra #{index}/{len(wv.index_to_key)} es {word}")

palabra #0/1193514 es <user>
palabra #1/1193514 es .
palabra #2/1193514 es :
palabra #3/1193514 es rt
palabra #4/1193514 es ,
palabra #5/1193514 es <repeat>
palabra #6/1193514 es <hashtag>
palabra #7/1193514 es <number>
palabra #8/1193514 es <url>
palabra #9/1193514 es !
palabra #10/1193514 es i
palabra #11/1193514 es a
palabra #12/1193514 es "
palabra #13/1193514 es the
palabra #14/1193514 es ?
palabra #15/1193514 es you
palabra #16/1193514 es to
palabra #17/1193514 es (
palabra #18/1193514 es <allcaps>
palabra #19/1193514 es <elong>


In [3]:
vec_king = wv['king']

In [4]:
vec_king

array([-0.77201  , -0.16548  ,  0.22263  , -0.56608  ,  0.15602  ,
       -0.050659 ,  0.076896 ,  0.90058  , -0.22829  , -0.083794 ,
       -0.0087308,  0.12425  , -3.6283   , -0.70631  ,  0.3391   ,
       -0.26866  ,  0.012886 ,  0.1314   ,  0.13072  ,  0.1594   ,
       -0.43884  ,  0.30631  , -0.51841  , -0.86402  ,  0.89706  ,
       -0.29222  ,  0.071633 , -0.7285   ,  0.47514  , -0.54581  ,
        0.37375  , -0.2815   , -0.82164  , -0.1245   ,  0.06561  ,
        0.2686   ,  0.12587  , -0.50189  ,  0.41322  , -0.40509  ,
       -0.88866  , -0.71627  , -0.010728 , -0.29513  ,  0.098062 ,
        0.47936  ,  0.49517  , -0.30246  ,  0.37465  ,  0.010619 ],
      dtype=float32)

Desafortunadamente, el modelo no puede inferir vectores para palabras desconocidas. Esta es una limitación de Word2Vec: si esta limitación le importa, consulte el modelo FastText.

In [23]:
wv["sanime"]

KeyError: "Key 'sanime' not present"

Continuando, Word2Vec admite varias tareas de similitud de palabras listas para usar. Puedes ver cómo la similitud (¿Que similitud será?) disminuye intuitivamente a medida que las palabras se vuelven cada vez menos similares.

In [27]:
pairs = [
    ('gato', 'perro'),   
    ('gato', 'tigre'),   
    ('gato', 'rana'),  
    ('gato', 'nube'),   
    ('gato', 'politica'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

'gato'	'perro'	0.85
'gato'	'tigre'	0.65
'gato'	'rana'	0.38
'gato'	'nube'	0.28
'gato'	'politica'	0.15


Podemos encontrar las palabras más similares de acuerdo a una medida de similaridad ([la similaridad del coseno](https://en.wikipedia.org/wiki/Cosine_similarity)):

In [32]:
pd.DataFrame(wv.most_similar(positive=['carro', 'camión'], topn=5), columns=["palabra", "similaridad"])

,palabra,similaridad
0,camion,0.847325
1,coche,0.836260
2,autobús,0.834142
3,helicóptero,0.820165
4,avión,0.792418


¿Cuál de estos no pertenece a la secuencia?

In [33]:
print(wv.doesnt_match(['fuego', 'agua', 'tierra', 'mar', 'aire', 'carro']))

carro


### Analogias

Se ha demostrado que los vectores de palabras *a veces* exhiben la capacidad de resolver analogías.

Como ejemplo, para la analogía "hombre: rey :: mujer: x" (léase: el hombre es al rey como la mujer es a x), ¿qué es x?

En la celda a continuación, se muestra cómo usar vectores de palabras para encontrar x usando la función `most_similar` de la [documentacion de GenSim](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similares). La función encuentra palabras que son más similares a las palabras en la lista "positiva" y más diferentes de las palabras en la lista "negativa" (mientras omite las palabras de entrada, que a menudo son las más similares). La respuesta a la analogía tendrá la mayor similitud del coseno (mayor valor numérico devuelto).

In [39]:
# esta es la analogia -- hombre : rey :: mujer : x
wv.most_similar(positive=['mujer', 'rey'], negative=['hombre'])

[('reina', 0.823491632938385),
 ('diana', 0.7247895002365112),
 ('victoria', 0.724120557308197),
 ('shakira', 0.7164205312728882),
 ('chica', 0.7134189009666443),
 ('luna', 0.6988048553466797),
 ('colombiana', 0.6971472501754761),
 ('canción', 0.6866007447242737),
 ('dulce', 0.6847524046897888),
 ('karina', 0.6814141869544983)]

#### Ejercicio 1
Cree una función denominada `analogia(a,b,c)` donde determine la analogía *a* : *b* :: *c* : *d*, es decir retorne a *d*, como en el ejemplo anterior. Pruebe la función con 5 analogías que se le ocurra (en inglés o en españól, pero no combinadas)

#### Ejercicio 2
Haga un listado de unas 20 palabras y grafiquelas en dos dimensiones (colocando su texto) junto con unas 200 palabras muestreadas aleatoriamente (sin texto). Use para reducir la dimensión PCA. Repita lo mismo usando UMAP.

-----

GenSim permite acceder a las palabras mediante diferentes formas sobre un objeto `KeyedVectors` (el wv de antes es uno). `.index_to_key`produce una lista con el vocabulario de forma ordenada, mientras `.key_to_index` produce un diccionario de la forma {palabra: index}.

In [56]:
print(wv.index_to_key[100:120])

['_', 'mi', 'can', '<sadface>', 'من', '♡', '´', 'he', 'con', 'they', 'now', 'go', '،', 'para', 'los', 'know', 'haha', 'good', 'tu', 'back']


## Entrene un nuevo modelo sobre un corpus

Trabajaremos de nuevo con el dataset de `progressive-tweet-sentiment.csv`, el cuál es un dataset pequeño que nos facilitará probar Word2vec. Sin embargo, los resultados pueden no ser tan buenos, dado que Word2vec es más potente cuando el corpus es más grande.

`progressive-tweet-sentiment`  tiene algunos tweets recopilados y categorizados en 4 clases: 'Legalization of Abortion', 'Hillary Clinton', 'Feminist Movement', 'Atheism'. 

In [60]:
df = pd.read_csv("data/progressive-tweet-sentiment.csv",encoding='latin-1')

In [61]:
df = df[["target", "tweet"]]
df.head()

,target,tweet
0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...
1,Legalization of Abortion,@rosaryrevival Lovely to use Glorious Mysterie...
2,Legalization of Abortion,@Niall250 good thing is that #DUP have consist...
3,Legalization of Abortion,"So, you tell me... is murder okay if the victi..."
4,Legalization of Abortion,@HillaryClinton Don't you mean to say (all chi...


#### Ejercicio 3. 
Realice procedimientos para preprocesar texto cómo: tokenizar, eliminar stopwords, stemming y lemmatization. Al final, el resultado de dicho procesamiento debe ser un texto (no una lista de palabras). Sugerencia: inicie y finalice con métodos de strings de python como `.join()` o `.split()`. (Esto ya lo han hecho en el taller anterior).

Nota: En NLP, a menudo agregamos tokens <START> y <END> para representar el principio y el final de oraciones, párrafos o documentos. En este caso, pueden colocar tokens `<START>` y `<END>` encapsulando cada documento, por ejemplo, "<START> All that glitters not gold <END>", y se puede incluir los tokens en el corpus completo. No es necesario hacer esto para el ejercicio, pero sería interesante.

In [62]:
#Debe quedar algo asi:

,target,tweet,text_clean
0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,thank another day life lord christian catholic...
1,Legalization of Abortion,@rosaryrevival Lovely to use Glorious Mysterie...,rosaryrevival lovely use glorious mystery east...
2,Legalization of Abortion,@Niall250 good thing is that #DUP have consist...,niall250 good thing dup consistently said murd...
3,Legalization of Abortion,"So, you tell me... is murder okay if the victi...",tell murder okay victim mentally disabled
4,Legalization of Abortion,@HillaryClinton Don't you mean to say (all chi...,hillaryclinton dont mean say child deserve cha...


### Entrada al modelo
Para entrenar un modelo en GenSim es importante adecuar el texto a una lista de sentencias (y cada sentencia una lista de tokens). Para corpus muy grandes, es mejor crear un iterador (una función que extraiga documento a documento para evitar llenar la memoria). 

Por ejemplo las tres primeras sentencias del dataset son:

[['thank', 'another', 'day', 'life', 'lord', 'christian', 'catholic', 'teamjesus'], ['rosaryrevival', 'lovely', 'use', 'glorious', 'mystery', 'eastertide', 'mark', 'season', 'prayforpersecutedchurch'], ['niall250', 'good', 'thing', 'dup', 'consistently', 'said', 'murder', 'wrong', 'sf', 'pro', 'murderstill']]


#### Ejercicio 4
Entrene un modelo usando `gensim.models.word2vec.Word2Vec`, partiendo de la siguiente configuración de parámetros.

In [644]:
vector_size = 50 # numero de elementos del vector que representa la palabra
min_count = 1 # Ignores all words with total frequency lower than this. 
workers = 3 # numero de cpu cores
sg = b # 0: CBOW, 1: skip-gram
window = 5 # Tamano de la ventana de contexto
sample = 1e-3 # tasa de submuestreo para terminos frecuentes

In [645]:
model = Word2Vec( #....

)

#### Ejercicio 5
Ahora ha obtenido un modelo de Word2vec en el cual tiene una representación embebida de cada palabra. Esta representación la puede extraer para cada palabra usando `model.wv.get_vector(palabra)`. Sin embargo, ¿Qué representación podemos obtener para cada tweet (documento o sentencia) a partir de todas las palabras? 

Cree una función que extraiga para cada tweet un representación vectorial única (un vector) y añada una nueva columna con esta representación. Sugerencia: una suma (pero será lo mejor?). El resultado es algo similar a la celda siguiente:

,target,tweet,text_clean,vectors
0,Legalization of Abortion,Thank you for another day of life Lord. #Chris...,thank another day life lord christian catholic...,"[0.0032098123, 0.0031128703, 0.0027382753, 0.0..."
1,Legalization of Abortion,@rosaryrevival Lovely to use Glorious Mysterie...,rosaryrevival lovely use glorious mystery east...,"[0.0029359048, 0.0028640802, 0.0026927434, 0.0..."
2,Legalization of Abortion,@Niall250 good thing is that #DUP have consist...,niall250 good thing dup consistently said murd...,"[0.00285275, 0.0033985889, 0.0021539323, 0.003..."
3,Legalization of Abortion,"So, you tell me... is murder okay if the victi...",tell murder okay victim mentally disabled,"[0.0013774268, 0.002832932, -0.0004417422, 0.0..."
4,Legalization of Abortion,@HillaryClinton Don't you mean to say (all chi...,hillaryclinton dont mean say child deserve cha...,"[0.003026036, 0.0016216192, 0.0023738334, 0.00..."


#### Ejercicio 6.
Lo que se ha generado antes son unas determinadas variables, para cada tweet, úselas para realizar una clasificación. Para esto divida en un conjunto de entrenamiento y uno de prueba (20%) con una semilla fija (42) y construya un clasificador (recomendado Random Forest). Evalúe el desempeño del clasificador en el conjunto de prueba con el accuracy.  El accuracy debe estar alrededor del 70%. Para lograrlo, tendrá que cambiar la longitud del vector, el ancho de la ventana, si es skip-gram o CBOW, el método de obtención de un sólo documento, etc.
